In [ ]:
import numpy as np
import pandas as pd
import nltk
import time
import openai
openai.api_key = "TYPE HERE YOUR API KEY"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Determination of language skills using NLTK

In [ ]:
# load data and make new columns
df = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df['languageSkills'] = ['' for i in range(len(df))]

# definition of keywords
relevant_words = ['english', 'hungarian', 'german', 'french', 'spanish', 'romanian', 'italian',
                  'angol', 'magyar', 'német', 'francia', 'spanyol', 'román', 'olasz']

# tokenization the jobDescriptions
for index, row in df.iterrows():
    tokens = nltk.word_tokenize(row['jobDescription'].lower())

    # definition of language skills
    for token in tokens:
        if token in relevant_words:
            if df.at[index, 'languageSkills']:
                df.at[index, 'languageSkills'] += ', '
            df.at[index, 'languageSkills'] += token

# print results
df[['companyName', 'languageSkills']]


## Determination of language skills using AI

In [ ]:
# Létrehozzuk az új dataframe-et
LI_PM_jobad = pd.read_csv('LI_PM_jobDesc.csv', delimiter = ',')
df2 = pd.DataFrame()
df2['companyName'] = LI_PM_jobad['companyName']
df2['jobDescription'] = LI_PM_jobad['jobDescription']
df2['languageSkills'] = ''

#iterate through the dataframe
for index, row in df2.iterrows():
    
    prompt_languageSkills = row['jobDescription'] + "/n --- /n This is a product manager job description.\
                            Define expectations related to language skills of applicant.\
                            Don't write complete sentences, just list the languages, the level of language knowledge,\
                            and whether the use of the language is expected in written or spoken.\
                            Template: {language}, {level}, written or spoken or both.\
                            Choose the level from this list: Beginner, Intermediate, Advanced, Proficient, Native!\
                            Answer the question as truthfully as possible using the provided text.\
                            If the expected language skills cannot be clearly determined based on\
                            the description, use just this value: 'Unknown' and don't write anything else."
        
    response_languageSkills = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = [
            {"role": "system", "content": "You are a helpful data science assistant."},
            {"role": "user", "content": prompt_languageSkills}],
        max_tokens=50,
        temperature=0)
    
    message_languageSkills = response_languageSkills['choices'][0]['message']['content'].strip()

    df2.loc[index, 'languageSkills'] = message_languageSkills
    
    time.sleep(2)
        
    #if index % 10 == 0:
    print(f"Processed row {index} : {message_languageSkills}")
        
#print results
df2[['companyName', 'languageSkills']]

## Further analysis with chatGPT

At first, I classified the text fragments collected by the AI from the 212 job advertisements into 2 large groups. Then, I copied these groups one by one to ChatGPT and asked it to help me with the analysis using the following prompt:

"I will show some job advertisement details about language skills given by employers - from now on, I will refer to this group of advertisements as jobAd_n.
Tasks:
1) Combine similar language skills in the jobAd_n advertisement group!
2) Then, collect the 15 most frequently occurring language skills from this jobAd_n group!
3) Finally, make a table of how many advertisements appeared in the jobAd_n group!"

Then, I saved the table created by ChatGPT for myself and performed the same analysis with all advertisement groups. Finally, I copied the tables created after all 2 groups and asked ChatGPT to combine similar language skills and make a last combined table.